In [1]:
from torch.utils.data.dataloader import DataLoader

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

import torchsummary

from utils import extract_features

In [2]:
train_dir = './train_another'
validation_dir = './validation_another'
test_dir = './test_another'

train = ImageFolder(train_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

validation = ImageFolder(validation_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

trainLoader = DataLoader(train, 20, shuffle = True, num_workers = 4, pin_memory = True)
validationLoader = DataLoader(validation, 20, shuffle = True, num_workers = 4, pin_memory = True)

In [3]:
# ---- Importation d'un modèle ----
model = torchvision.models.resnet18(pretrained=True)
torchsummary.summary(model, (3,150,150))
#On récupère le nombre output du modèle

C:\Users\Asus\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Asus\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 75, 75]           9,408
       BatchNorm2d-2           [-1, 64, 75, 75]             128
              ReLU-3           [-1, 64, 75, 75]               0
         MaxPool2d-4           [-1, 64, 38, 38]               0
            Conv2d-5           [-1, 64, 38, 38]          36,864
       BatchNorm2d-6           [-1, 64, 38, 38]             128
              ReLU-7           [-1, 64, 38, 38]               0
            Conv2d-8           [-1, 64, 38, 38]          36,864
       BatchNorm2d-9           [-1, 64, 38, 38]             128
             ReLU-10           [-1, 64, 38, 38]               0
       BasicBlock-11           [-1, 64, 38, 38]               0
           Conv2d-12           [-1, 64, 38, 38]          36,864
      BatchNorm2d-13           [-1, 64, 38, 38]             128
             ReLU-14           [-1, 64,

In [4]:
# ---- Extraction des features ----
train_features, train_labels = extract_features(trainLoader, model, 10000, 1000)
test_features, test_labels = extract_features(validationLoader, model, 2000, 1000)

#Taille (Normalement n_sample, n_features)
train_features.shape

#Classes
train_labels

array([1., 0., 0., ..., 0., 1., 0.])

In [5]:
# ---- Entrainement d'un modèle XGB a partir des features de sortie du modèle resnet18 ----
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(train_features, train_labels)
y_pred = classifier.predict(test_features)
accuracy_score(test_labels, y_pred)

0.9135

In [6]:
# ---- Random forest ----
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(train_features, train_labels)
y_pred = classifier.predict(test_features)
accuracy_score(test_labels, y_pred)

0.8875

In [7]:
# ---- LightGBM ----
import lightgbm as lgb
classifier = lgb.LGBMClassifier()
classifier.fit(train_features, train_labels)
y_pred = classifier.predict(test_features)
accuracy_score(test_labels, y_pred)

0.914